# **Signal To Spike Conversion** - Pre-Processing of the SEEG Signal (2/2)
This notebook presents the **pre-processing stage 2** the SEEG signal goes through before being fed to the SNN. The pre-processing stages are as follows:
1. **Filtering**: The SEEG signal is bandpass filtered to remove noise and artifacts. The bandpass filter is designed using the Butterworth filter and, since we are working with *iEEG*, the signal is filtered in the ripples and FR bands. The co-occurrence of HFOs in both bands is an optimal prediction of post-surgical seizure freedom by defining an optimal "HFO area" or EZ zone.
2. **Signal-to-Spike Conversion**: To interface and communicate with the silicon neurons in the SNN, the SEEG signal must be converted to spikes.

### Check WD (change if necessary) and file loading

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

PATH_TO_FILE = '' # 'src/hfo/'  # This is needed if the WD is not the same as the file location

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


## Add the parent directory to the path to import data from the `filter` folder

In [2]:
# Add the parent directory to the path to be able to import data from the filter folder
import sys
sys.path.append('../../')

## Checkpoint 1

Right now, we have the filtered SEEG signal in both the ripple and FR bands. The next step is to convert the signal to spikes.

## Let's first load the filtered SEEG signal and the corresponding markers
This notebook only converts 1 SEEG channel to spikes. Therefore, the `.npy` file must contain a single channel.

In [2]:
import numpy as np
import math
from utils.io import preview_np_array

# Load the filtered seeg signal in the ripple band
ripple_seeg_file_name = "synthetic/seeg_filtered_subset_90-119_ripple_band.npy"
ripple_band_seeg = np.load(f"{PATH_TO_FILE}filter/results/data/{ripple_seeg_file_name}")

# Remove the extra inner dimension
ripple_band_seeg = np.squeeze(ripple_band_seeg)

preview_np_array(ripple_band_seeg, "Ripple Band SEEG", edge_items=3)

Ripple Band SEEG Shape: (245760,).
Preview: [1.84829940e-04 1.37975809e-03 3.92964380e-03 ... 1.23691538e+00
 7.54000855e-01 1.73992494e-01]


In [3]:
# Load the filtered seeg signal in the fast ripple band
fr_seeg_file_name = "seeg_filtered_subset_90-119_fr_band.npy"
fr_band_seeg = np.load(f"{PATH_TO_FILE}data/{fr_seeg_file_name}")

# Remove the extra inner dimension
fr_band_seeg = np.squeeze(fr_band_seeg)

preview_np_array(fr_band_seeg, "FR Band SEEG", edge_items=3)

FR Band SEEG Shape: (245760,).
Preview: [ 0.00095524  0.00310525 -0.00478473 ... -0.49596476 -0.2753618
  0.28661303]


In [4]:
# Load the annotated events
markers_file_name = "seeg_filtered_subset_90-119_markers.npy"
markers = np.load(f"{PATH_TO_FILE}data/{markers_file_name}")

# Remove the extra inner dimension
markers = np.squeeze(markers)

preview_np_array(markers, "markers", edge_items=3)

markers Shape: (226,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Spike+Ripple', 116216.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Ripple', 119000.  , 0.)]


## Define Global Parameters of the Experiment

In [5]:
# TODO: This could be in the utils?

sampling_rate = 2048    # 2048 Hz
input_duration = 120 * (10**3)    # 120000 ms or 120 seconds
num_samples = ripple_band_seeg.shape[0]    # 2048 * 120 = 245760

x_step = 1/sampling_rate * (10**3)  # 0.48828125 ms

## Signal-to-Spike Conversion
The signal can be converted to spikes in different ways. First, we will try a method where **two spike trains are generated from the filtered signal**:
- **UP Spike Train**: The spikes are generated based on an increase of the signal's amplitude. The spikes are generated when the signal crosses a certain threshold defined by `threshold_up`.
- **DOWN Spike Train**: The spikes are generated based on a decrease of the signal's amplitude. The spikes are generated when the signal crosses a certain threshold defined by `threshold_down`.

The spike trains are generated by comparing the amount of change in the signal since the last time a spike was generated (UP or DOWN). If the positive/negative amplitude change is greater than the defined threshold, the algorithm stores the current timestep in the respective spike train and takes the new amplitude as the reference for the next comparison.

Another important aspect of this algorithm is to model the time that silicon neurons need before they can generate another spike. Both in hardware and software, we call this time `refractory_period`.

### Configurable Parameters for the Signal-to-Spike Conversion
- `threshold_up`: The threshold for the UP spike train.
- `threshold_down`: The threshold for the DOWN spike train.

The accuracy of this algorithm is heavily dependent on the choice of these parameters. To find the optimal values, we can perform a ***baseline detection*** to determine the optimal spike generation threshold automatically for the signal conversion.

**As a first solution, we set these values manually to have a working prototype. Later, we will find the optimal values and compare the results of both methods.**

In [74]:
# Define variables of the Signal to Spike Conversion Manually
ripple_threshold_up = 5   # Threshold for the UP spike detection (in μV)
ripple_threshold_down = -5 # Threshold for the DOWN spike detection (in μV)

fr_threshold_up = 3   # Threshold for the UP spike detection (in μV)
fr_threshold_down = -3 # Threshold for the DOWN spike detection (in μV)

In [75]:
# Interactive Plot for the HFO detection
# bokeh docs: https://docs.bokeh.org/en/2.4.1/docs/first_steps/first_steps_1.html

from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

# Define the x and y values
# Should the first input start at 0 or x_step?
# TODO: is it okay to create a range with floats?
x = [val for val in np.arange(x_step, input_duration + x_step, x_step)] 

In [76]:
from hfo.signal_to_spike.signal_to_spike import signal_to_spike, SignalToSpikeParameters

# Convert the filtered ripple signal to spikes
ripple_spike_trains = signal_to_spike(
    SignalToSpikeParameters(
        signal=ripple_band_seeg, times=np.array(x),
        threshold_up=ripple_threshold_up, threshold_down=ripple_threshold_down,
        # refractory_period=0.002, interpolation_factor=1
        )
)

np.set_printoptions(edgeitems=5)
print("Ripple UP Spike Train shape: ", ripple_spike_trains.up.shape, "Preview: ", ripple_spike_trains.up)
print("Ripple DOWN Spike Train shape: ", ripple_spike_trains.down.shape, "Preview: ", ripple_spike_trains.down)

Ripple UP Spike Train shape:  (2410,) Preview:  [   944.82421875    951.171875      953.125         981.93359375
    993.1640625  ... 119083.0078125  119084.47265625 119092.28515625
 119093.75       119106.4453125 ]
Ripple DOWN Spike Train shape:  (2410,) Preview:  [   947.265625      948.73046875    956.0546875     979.00390625
    989.2578125  ... 119075.68359375 119086.9140625  119087.890625
 119089.35546875 119100.09765625]


In [77]:
# Convert the filtered FR signal to spikes
fr_spike_trains = signal_to_spike(
    SignalToSpikeParameters(
        signal=fr_band_seeg, times=np.array(x),
        threshold_up=fr_threshold_up, threshold_down=fr_threshold_down,
        # refractory_period=0.002, interpolation_factor=1   # TODO: Add refractory period
        )
)

np.set_printoptions(edgeitems=5)
print("Fast Ripple UP Spike Train shape: ", fr_spike_trains.up.shape, "Preview: ", fr_spike_trains.up)
print("Fast Ripple DOWN Spike Train shape: ", fr_spike_trains.down.shape, "Preview: ", fr_spike_trains.down)

Fast Ripple UP Spike Train shape:  (2111,) Preview:  [  1001.953125     1002.44140625   1005.37109375   1005.859375
   1008.30078125 ... 116781.73828125 116784.66796875 116785.15625
 116789.55078125 116793.45703125]
Fast Ripple DOWN Spike Train shape:  (2050,) Preview:  [  1000.9765625    1003.90625      1006.8359375    1007.32421875
   1010.7421875  ... 116782.71484375 116783.203125   116786.62109375
 116791.015625   116794.43359375]


## Visualize the Spike Trains
Let's plot the generated spike trains via a raster plot.

In [78]:
from utils.raster_plot import create_raster_fig

# ------------------------------------------------------------------------------- #
# ------- Create the raster plot for the Ripple UP and DOWN spike trains -------- #
# ------------------------------------------------------------------------------- #

# Create a list containing the x values for the raster plot.
ripple_raster_x = np.concatenate((ripple_spike_trains.up, ripple_spike_trains.down), axis=0)

# Create a list containing the y values for the raster plot.
# The UP spike train will be represented by 1s and the DOWN spike train by 0s
ripple_raster_y = [1 for _ in range(len(ripple_spike_trains.up))] + [0 for _ in range(len(ripple_spike_trains.down))]

ripple_train_raster = create_raster_fig("Ripple UP and DOWN spike events", "Time (ms)", "Channel", ripple_raster_x, ripple_raster_y)

In [79]:
import bokeh.plotting as bplt
showRasterPlot = True

# Plot the raster plot for the Ripple spike trains
if showRasterPlot:
    bplt.show(ripple_train_raster)

In [80]:
# ------------------------------------------------------------------------------- #
# ------- Create the raster plot for the Fast Ripple UP and DOWN spike trains -------- #
# ------------------------------------------------------------------------------- #

# Create a list containing the x values for the raster plot.
fr_raster_x = np.concatenate((fr_spike_trains.up, fr_spike_trains.down), axis=0)

# Create a list containing the y values for the raster plot.
# The UP spike train will be represented by 1s and the DOWN spike train by 0s
fr_raster_y = [1 for _ in range(len(fr_spike_trains.up))] + [0 for _ in range(len(fr_spike_trains.down))]

fr_train_raster = create_raster_fig("Fast Ripple UP and DOWN spike events", "Time (ms)", "Channel", fr_raster_x, fr_raster_y)

In [81]:
# Plot the raster plot for the Fast Ripple spike trains
if showRasterPlot:
    bplt.show(fr_train_raster)

## Export the Spike Trains to CSV Files for the Lava SNN

We have successfully converted the SEEG signal to spikes. The next step is to feed these spikes to the SNN for Ripple and Fast Ripple detection.

For this, we will create a file for each type of spike train (UP and DOWN). I'm not sure if we should join the spikes of both bands in a single file or keep them separate.

In [82]:
# Create a csv file with the spike train data
import csv

def write_spike_train_to_csv(file_name, spike_train, channel_idx):
    """
    This function writes the spike train to a csv file.
    @file_name (str): Name of the file to be created.
    @spike_train (np.ndarray): Array with the spike train data.
    @channel_idx (int): Index of the channel that generated the spike train. (According to the original data)
    """
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["time", "channel_idx"])
        for spike_time in spike_train:
            writer.writerow([spike_time, channel_idx])

In [85]:
WRITE_RIPPLE_CSV_FILES = False
WRITE_FR_CSV_FILES = False

In [86]:
selected_ch_idx = -1    # TODO: Should we really send the channel?

if WRITE_RIPPLE_CSV_FILES:
    # Create the csv file for the Ripple UP spike train
    ripple_up_file_name = f"{PATH_TO_FILE}snn/data/custom_subset_90-119/ripple_up_spike_train_5.csv"
    write_spike_train_to_csv(ripple_up_file_name, ripple_spike_trains.up, selected_ch_idx)

    # Create the csv file for the Ripple DOWN spike train
    ripple_down_file_name = f"{PATH_TO_FILE}snn/data/custom_subset_90-119/ripple_down_spike_train_-5.csv"
    write_spike_train_to_csv(ripple_down_file_name, ripple_spike_trains.down, selected_ch_idx)

In [87]:
if WRITE_FR_CSV_FILES:
    # Create the csv file for the Fast Ripple UP spike train
    fr_up_file_name = f"{PATH_TO_FILE}snn/data/custom_subset_90-119/fr_up_spike_train_3.csv"
    write_spike_train_to_csv(fr_up_file_name, fr_spike_trains.up, selected_ch_idx)

    # Create the csv file for the Fast Ripple DOWN spike train
    fr_down_file_name = f"{PATH_TO_FILE}snn/data/custom_subset_90-119/fr_down_spike_train_-3.csv"
    write_spike_train_to_csv(fr_down_file_name, fr_spike_trains.down, selected_ch_idx)

## Generate the input files for the SNN

### Is the SNN going to detect the HFO events in windows? Or do we take it as a continous input?
**If so**: The SNN is going to detect HFO events in windows. Therefore, the input to the network must be organized in windows of a certain size. The size of the window is a hyperparameter that can be tuned to improve the performance of the network.

I think windowing makes more sense when we are learning with an ANN. Since we want real-time detection, feeding a continous input makes more sense.

### Let's assume we do NOT need to window the input
In this case, our input will simply be a continous stream of spikes. We can feed the spikes to the SNN in real-time. At each timestep, the SNN will receive 2 binary inputs (UP and DOWN spikes) indicating the presence of a spike in the respective spike train.

In [88]:
# Create a numpy array that will store the input (2D)
snn_input = np.zeros((num_samples, 2))  # 2 columns: UP and DOWN spike trains

# ---------------------------------------------------------------------------------- #
# -------- Select the Spike Trains to be used as input for the SNN ----------------- #
# ---------------------------------------------------------------------------------- #
selected_up_spikes = ripple_spike_trains.up
selected_down_spikes = ripple_spike_trains.down

# Iterate the time steps of the recording and check if there are spikes in the selected spike trains at each timestep
curr_up_idx = 0
curr_down_idx = 0
for (idx, time_step) in enumerate(x):
    # Check if an UP spike occurs at this time step
    if curr_up_idx < len(selected_up_spikes) and selected_up_spikes[curr_up_idx] <= time_step:
        snn_input[idx][0] = 1   # Mark the UP spike in the input array
        curr_up_idx += 1    # Move to the next spike in the UP spike train
    # Check if a DOWN spike occurs at this time step
    elif curr_down_idx < len(selected_down_spikes) and selected_down_spikes[curr_down_idx] <= time_step:
        snn_input[idx][1] = 1   # Mark the DOWN spike in the input array
        curr_down_idx += 1  # Move to the next spike in the DOWN spike train
    
    if curr_up_idx >= len(selected_up_spikes) and curr_down_idx >= len(selected_down_spikes):
        # All the spikes have been added to the input array
        break

np.set_printoptions(edgeitems=5)
print("snn_input: ", snn_input.shape, "Preview:", snn_input)

snn_input:  (245760, 2) Preview: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [89]:
# Export the input array to a numpy file
EXPORT_INPUT = True
if EXPORT_INPUT:
    input_file_name = f"{PATH_TO_FILE}snn/data/custom_subset_90-119/snn_input_ripple_5_-5.npy"
    np.save(input_file_name, snn_input)

    # Export to CSV for visualization purposes
    with open(f"{PATH_TO_FILE}snn/data/custom_subset_90-119/snn_input_ripple_5_-5.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["time", "up_spike", "down_spike"])
        for (idx, time_step) in enumerate(x):
            writer.writerow([time_step, snn_input[idx][0], snn_input[idx][1]])

## Generate the Ground Truth File for the SNN (Target)

Extract the markers from the selected channel

In [93]:
# The target output for the SNN must have the same length as the input.
target_np = np.zeros((num_samples))  # 1 column: 0/1 for the output classes (No Event, Ripple/Fast Ripple or both)
# TODO: Could have more than 2 classes to differentiate the labels

# Get the markers for the selected channel
# Each marker has the following keys:   position, label, and duration
selected_ch_markers = markers
preview_np_array(selected_ch_markers, "Selected Channel Markers")

Selected Channel Markers Shape: (226,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ('Ripple+Fast-Ripple',   4134.77, 0.)
 ('Fast-Ripple',   4774.41, 0.) ...
 ('Spike+Ripple+Fast-Ripple', 115019.  , 0.)
 ('Spike+Fast-Ripple', 115777.  , 0.) ('Spike+Ripple', 116216.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Ripple', 119000.  , 0.)]


### Convert to the format that the learnable SNN (Slayer) can read

#### Fill the target numpy array with 1s where the HFOs are present

In [94]:
from utils.input import label_has_hfo_event

# Iterate the time steps of the recording and check if there is an annotated event at each timestep
curr_markers_idx = 0
for (idx, time_step) in enumerate(x):
    # Check if an event occurs at this time step
    if curr_markers_idx < len(selected_ch_markers) and selected_ch_markers[curr_markers_idx]['position'] <= time_step:
        # If the label has an HFO event, mark it as 1 in the target array
        if label_has_hfo_event(selected_ch_markers[curr_markers_idx]['label']):
            target_np[idx] = 1   # Mark the Labelled event in the target array
        
        curr_markers_idx += 1    # Move to the next annotated event
    
    if curr_markers_idx >= len(selected_ch_markers):
        # All the spikes have been added to the input array
        break

In [95]:
preview_np_array(target_np, "target_np")

target_np Shape: (245760,).
Preview: [0. 0. 0. 0. 0. ... 0. 0. 0. 0. 0.]


## Export the target file to a numpy file

In [97]:
# Export the target array to a numpy file
EXPORT_TARGET = True
if EXPORT_TARGET:
    target_file_name = f"{PATH_TO_FILE}snn/data/custom_subset_90-119/ground_truth.npy"
    np.save(target_file_name, target_np)